In [ ]:
# نصب فقط یک‌بار
!pip install mplfinance

In [ ]:
import requests
import pandas as pd
import numpy as np
import mplfinance as mpf
from datetime import datetime, timedelta

# === Token & API ===
access_token = "YOUR_ACCESS_TOKEN_HERE"  # 👈 توکن واقعی رو جایگزین کن

headers = {
    "Authorization": f"Bearer {access_token}",
    "Content-Type": "application/json",
    "accept": "application/json"
}

url = "https://api.topstepx.com/api/History/retrieveBars"
payload = {
    "contractId": "CON.F.US.MNQ.U25",
    "live": True,
    "startTime": "2025-06-24T00:00:00Z",
    "endTime": "2025-06-25T00:00:00Z",
    "unit": 2,
    "unitNumber": 5,
    "limit": 100,
    "includePartialBar": False
}

# === Request ===
response = requests.post(url, json=payload, headers=headers)
bars = response.json()

if "bars" in bars and bars["bars"]:
    df = pd.DataFrame(bars["bars"])
    df.rename(columns={"t": "Date", "o": "Open", "h": "High", "l": "Low", "c": "Close", "v": "Volume"}, inplace=True)
    df["Date"] = pd.to_datetime(df["Date"])
    df.set_index("Date", inplace=True)

    # Heikin Ashi
    ha_df = df.copy()
    ha_df["HA_Close"] = (ha_df["Open"] + ha_df["High"] + ha_df["Low"] + ha_df["Close"]) / 4
    ha_open = [(ha_df["Open"].iloc[0] + ha_df["Close"].iloc[0]) / 2]
    for i in range(1, len(ha_df)):
        ha_open.append((ha_open[i - 1] + ha_df["HA_Close"].iloc[i - 1]) / 2)
    ha_df["HA_Open"] = ha_open
    ha_df["HA_High"] = ha_df[["High", "HA_Open", "HA_Close"]].max(axis=1)
    ha_df["HA_Low"] = ha_df[["Low", "HA_Open", "HA_Close"]].min(axis=1)

    # Moving Averages
    ema_fast = ha_df["Close"].ewm(span=9).mean()
    ema_slow = ha_df["Close"].ewm(span=29).mean()
    ema_trend = ha_df["Close"].ewm(span=100).mean()

    ha_df["EMA_Fast"] = ema_fast
    ha_df["EMA_Slow"] = ema_slow
    ha_df["EMA_Trend"] = ema_trend

    # Signal Conditions
    ha_df["Bull_Cross"] = (ema_fast > ema_slow) & (ema_fast.shift(1) <= ema_slow.shift(1))
    ha_df["Bear_Cross"] = (ema_fast < ema_slow) & (ema_fast.shift(1) >= ema_slow.shift(1))
    ha_df["Trend_Up"] = ha_df["Close"] > ema_trend
    ha_df["Trend_Down"] = ha_df["Close"] < ema_trend
    ha_df["HA_Bullish"] = ha_df["HA_Close"] > ha_df["HA_Open"]
    ha_df["HA_Bearish"] = ha_df["HA_Close"] < ha_df["HA_Open"]

    # Distance Filter
    min_dist = 0.15
    max_dist = 0.55
    ha_df["Dist_Long"] = (abs(ha_df["HA_High"] - ema_trend) / ema_trend) * 100
    ha_df["Dist_Short"] = (abs(ha_df["HA_Low"] - ema_trend) / ema_trend) * 100
    ha_df["Long_Entry_OK"] = ha_df["Dist_Long"].between(min_dist, max_dist)
    ha_df["Short_Entry_OK"] = ha_df["Dist_Short"].between(min_dist, max_dist)

    # Final Signal
    ha_df["Signal_Long"] = ha_df["Bull_Cross"] & ha_df["Trend_Up"] & ha_df["HA_Bullish"] & ha_df["Long_Entry_OK"]
    ha_df["Signal_Short"] = ha_df["Bear_Cross"] & ha_df["Trend_Down"] & ha_df["HA_Bearish"] & ha_df["Short_Entry_OK"]

    last_row = ha_df.iloc[-1]
    signal = "None"
    if last_row["Signal_Long"]:
        signal = "BUY"
    elif last_row["Signal_Short"]:
        signal = "SELL"
    print("✅ Signal:", signal)

    # Chart
    mpf.plot(df, type='candle', style='charles', title='MNQU25 - Candlestick Chart', ylabel='Price', volume=True)
else:
    print("❌ No data or invalid response")